In [41]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

In [42]:
df = pd.read_csv("../data/processed/mastodon_dataset_all.csv")

# Data Cleaning

Lets check if the tags are functioning right

In [43]:
df["tag"].unique()

array(['tc', 'cooking'], dtype=object)

In [44]:
df_tc = df[df["tag"] == "tc"]
df_tc["tag"].unique()

array(['tc'], dtype=object)

In [45]:
df_ck = df[df["tag"] == "cooking"]
df_ck["tag"].unique()

array(['cooking'], dtype=object)

Now lets see if we have we have a proper sum of rows

In [46]:
df.shape

(7486, 18)

In [47]:
df_ck.shape

(3385, 18)

In [48]:
df_tc.shape

(4101, 18)

3385+4101=7486

All good!

Would be handy to have a field to work with just dates for our analysis

In [49]:
df["timestamp"].dtype

dtype('O')

we have to convert this datatype

In [50]:
df["timestamp_dt"] = pd.to_datetime(df["timestamp"], format="mixed", utc=True)

In [51]:
df["timestamp_dt"].dtype

datetime64[ns, UTC]

In [52]:
df_clean_date = df

In [53]:
df_clean_date["timestamp"] = df_clean_date["timestamp_dt"]

In [54]:
df_clean_date["timestamp"].dtype

datetime64[ns, UTC]

In [55]:
df_clean_date = df_clean_date.drop(columns=["timestamp_dt"])

In [56]:
# final check
df_clean_date["timestamp"].dtype

datetime64[ns, UTC]

for less granularity lets have a column with just the date

In [57]:
df_clean_date["date"] = df_clean_date["timestamp"].dt.date


In [58]:
df = df_clean_date

In [59]:
df.shape

(7486, 19)

In [60]:
df.columns

Index(['tag', 'timestamp', 'weekday', 'hour', 'toot_id', 'toot_url',
       'language', 'visibility', 'content', 'username', 'display_name', 'acct',
       'followers_count', 'following_count', 'statuses_count', 'replies_count',
       'reblogs_count', 'favourites_count', 'date'],
      dtype='object')

In [61]:
df = df[['tag', 'timestamp', 'date', 'weekday', 'hour', 'toot_id', 'toot_url',
       'language', 'visibility', 'content', 'username', 'display_name', 'acct',
       'followers_count', 'following_count', 'statuses_count', 'replies_count',
       'reblogs_count', 'favourites_count']]

# Analysis

In [62]:
dup = df[df.duplicated(subset='content', keep=False)]


In [63]:
dup['content'].str.len().value_counts().sum()

np.int64(306)

In [64]:
df['toot_id'].duplicated().sum()

np.int64(1)

In [65]:
df = df.drop_duplicates(subset='toot_id')

In [66]:
df.shape

(7485, 19)

## Posting patterns

### Posting distribution - general view

In [67]:
import plotly.express as px


fig = px.density_heatmap(
    df,
    x='weekday',
    y='hour',
    z='toot_id',
    histfunc='count',
    nbinsy=24
)

order = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']

fig.update_xaxes(categoryorder='array', categoryarray=order)
fig.update_xaxes(categoryorder='array', categoryarray=order)


fig.show()

### Posting distribution - duplicates

In [68]:
fig = px.density_heatmap(
    dup,
    x='weekday',
    y='hour',
    z='toot_id',
    histfunc='count',
    nbinsy=24
)
order = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']

fig.update_xaxes(categoryorder='array', categoryarray=order)

fig.show()

It is very clear that the dataframe of duplicates tends to focus on the most popular hours respective to the general posting pattern, leaving almost empty the rest of the slots.

Therefore the suspicion of this part of the dataset being ruled by bots is solid

### Excluding Duplicates

To make sure, lets compare now with the whole dataset excluding the duplicates

In [69]:
df_unique_content = df.drop_duplicates(subset='content')

In [70]:
fig = px.density_heatmap(
    df_unique_content,
    x='weekday',
    y='hour',
    z='toot_id',
    histfunc='count',
    nbinsy=24
)
order = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']

fig.update_xaxes(categoryorder='array', categoryarray=order)

fig.show()

It has very similar patterns to the global df, so evidence that changes happen exclusively on the dataframe of duplicates is quite clear now. Bot activity quite probable

This is explained because Mastodon is a federated social network, with independent servers. A same user can register in several servers, and moderation is more difficult, making it easier for automated content to survive

## Comparison of duplicates between tags

### What is the proportion of duplicated content in each of the tags?

In [71]:
# we do a new column to flag that will make easier plotting later

df["is duplicated"] = df.duplicated(subset="content", keep=False)

In [72]:
# fist look at possible differences

df_dup_count = df.groupby(["tag", "is duplicated"])["toot_id"].count().reset_index()
df_dup_count


,tag,is duplicated,toot_id
0,cooking,False,3258
1,cooking,True,126
2,tc,False,3923
3,tc,True,178


In [73]:
df_dup_count["proportion"] = (df_dup_count["toot_id"] / df_dup_count.groupby("tag")["toot_id"].transform("sum"))
df_dup_count

,tag,is duplicated,toot_id,proportion
0,cooking,False,3258,0.962766
1,cooking,True,126,0.037234
2,tc,False,3923,0.956596
3,tc,True,178,0.043404


In [74]:
fig = px.bar(df_dup_count, x="tag", y="proportion", barmode="group", color="is duplicated")

fig.show()

We have around the same amount of duplicated posts on each tag, maybe we could do more comparisons between tags, since the main difference seems to appear regarding timing of posting patterns. Lets analyze this time the posting distribution of unique content, for the 2 topics

In [75]:
df_unique_content_tc = df_unique_content[df_unique_content["tag"] == "tc"]

In [76]:
df_unique_content_ck = df_unique_content[df_unique_content["tag"] == "cooking"]

### Posting schedule on topic Tech

In [77]:
fig = px.density_heatmap(
    df_unique_content_tc,
    x='weekday',
    y='hour',
    z='toot_id',
    histfunc='count',
    nbinsy=24
)
order = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']

fig.update_xaxes(categoryorder='array', categoryarray=order)

fig.show()

### Posting schedule on topic Cooking

In [78]:
fig = px.density_heatmap(
    df_unique_content_ck,
    x='weekday',
    y='hour',
    z='toot_id',
    histfunc='count',
    nbinsy=24
)
order = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']

fig.update_xaxes(categoryorder='array', categoryarray=order)

fig.show()

## Statistical Analysis

### Explaining presence of ducplicated content. Is repetitive content penalized on a federated social network?

lets consider engagement the sum of the following variables. to follow consistency with the approach in the whole team

In [79]:
df['engagement'] = df["replies_count"] + df["reblogs_count"] + df["favourites_count"]

In [81]:
df.groupby('is duplicated')['engagement'].describe()

,count,mean,std,min,25%,50%,75%,max
is duplicated,,,,,,,,
False,7181.0,3.254700,26.296793,0.0,0.0,1.0,2.0,2001.0
True,304.0,1.118421,2.143173,0.0,0.0,1.0,1.0,18.0


means are quite different, same median at 1... in social media there is a tendency of heavy tails... some people with great influence got most of the traffic and interactions.
Lets do further testing

### Mann–Whitney U test

Reasons for this test:

- Engagement is discrete
- Big variances
- Indicators of heavy-tail: max too far from the mean
- Different variances
- The Mann–Whitney U test is used due to non-normal engagement distributions.

Hypothesis testing:

- H0: probability of engagement of duplicates being the same as engagement of unique posts
- H1: probability of engagement of duplicates being distinct as engagement of unique posts


In [82]:
from scipy.stats import mannwhitneyu

dup_eng = df["engagement"][df["is duplicated"] ==True]
unique_eng = df["engagement"][df["is duplicated"] ==False]

stat, p = mannwhitneyu(dup_eng, unique_eng, alternative="two-sided")


In [83]:
stat

np.float64(807098.0)

In [84]:
p

np.float64(5.248214381135188e-16)

There is enough evidence to reject H0, the engangement of unique content and repetitive content seems to be statistically different

## How duplicated content and followers affects engagement

In [85]:
# we saw before that engagement is heavy tailed, therefore changing to logarithmic scale we compress the tail and reduce impact of outliers

df['log_engagement'] = np.log1p(df['engagement'])
df['is duplicated'] = df['is duplicated'].astype(int)

In [86]:
import statsmodels.api as sm

X = df[['is duplicated', 'followers_count']]
X = sm.add_constant(X)
y = np.log1p(df['engagement'])

model = sm.OLS(y, X).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:             engagement   R-squared:                       0.140
Model:                            OLS   Adj. R-squared:                  0.140
Method:                 Least Squares   F-statistic:                     608.7
Date:                Sun, 22 Feb 2026   Prob (F-statistic):          1.17e-245
Time:                        17:57:31   Log-Likelihood:                -8230.3
No. Observations:                7485   AIC:                         1.647e+04
Df Residuals:                    7482   BIC:                         1.649e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const               0.7737      0.009     

- R-squared apparently quite small at 0.14, but acceptable in social behavior
- p-value is close to 0, so its beta is statiscically significative

### We can conclude our first suspicion, and ensure that duplicating content doesnt help at all to obtain more engagement, more like the contrary since its coefficient is negative

In [87]:
beta = model.params['is duplicated']
percent_effect = (np.exp(beta) - 1) * 100

percent_effect

np.float64(-23.848066695709292)

since we have a logarithmic scale, that is the way to interpret beta for "is duplicated"

maintaining follower count the same, duplicated posts receive around 24% less engagement that unique ones